# Irrigation Water Requirement Prediction

**Background & Goals:** Nowadays, Farming accounts for 70% of the water consumed and most of its wasteful use. Over irrigation will increases the amount of salt on the surface of soil due to evaporation. The accumulation of salts damages the soil fertility.Accoding to data collected by sensors(humidity, temperature, soil moisture, etc.), we predict how much water will be irrigated periodicity. In this way, we can make irrigation more controllable and accurate. 

## Data Pre-processing

We are going to pre-process weather dataset and irrigation water usage for training dataset 

### Weather dataset

We can get weather information of coleambelly from the Australian Meteorological Bureau website(http://www.bom.gov.au/climate/data/index.shtml), such as daily maximum temperature, daily minimum temperature, solar intensity, rainfall.   we are going to use weather data from 1999-2017,

In [1]:
import pandas as pd

In [2]:
def changeFormate(df):
    df['Date']= str(int(df['Month']))+ '/' + str(int(df['Day']))+'/'+str(int(df['Year']))
    return df['Date']

def changeFormate1(df):
    df['Month/Year']= str(int(df['Month']))+ '/'+str(int(df['Year']))
    return df['Month/Year']

def dataFilter(df):
    df=df[(df['Year']>=2010)&(df['Year']<=2017)]
    return df

In [3]:
def calculateEToi(df):
    if(df['Minimum temperature (Degree C)']>0):
        df['EToi']=(0.408*0.0023*df['Daily global solar exposure (MJ/m*m)']*(
        df['Maximum temperature (Degree C)']-df['Minimum temperature (Degree C)'])**(0.5))*((df['Maximum temperature (Degree C)']+df['Minimum temperature (Degree C)'])/2 +17.8)
    else:
        df['EToi']=(0.408*0.0023*df['Daily global solar exposure (MJ/m*m)']*(
        df['Maximum temperature (Degree C)']+df['Minimum temperature (Degree C)'])**(0.5))*((df['Maximum temperature (Degree C)']+df['Minimum temperature (Degree C)'])/2 +17.8)
    return df['EToi']

Max temperature dataset

In [4]:
maxTemDf=pd.read_csv('dataset/maxtem.csv') 
maxTemDf['Date'] =maxTemDf.apply(lambda x: changeFormate(x),axis=1)
maxTemDf=dataFilter(maxTemDf)
toDrop=['Year','Month','Day']
maxTemDf.drop(toDrop,axis=1,inplace=True)

Min temperature dataset

In [5]:
minTemDf=pd.read_csv('dataset/mintem.csv') 
minTemDf['Date'] =minTemDf.apply(lambda x: changeFormate(x),axis=1)
minTemDf=dataFilter(minTemDf)
toDrop=['Year','Month','Day']
minTemDf.drop(toDrop,axis=1,inplace=True)

Rainfall dataset

In [6]:
rainfallDf=pd.read_csv('dataset/rainfall.csv') 
rainfallDf['Date'] =rainfallDf.apply(lambda x: changeFormate(x),axis=1)
rainfallDf=dataFilter(rainfallDf)
toDrop=['Year','Month','Day']
rainfallDf.drop(toDrop,axis=1,inplace=True)

Radiation dataset

In [7]:
solarDf=pd.read_csv('dataset/solar.csv') 
solarDf['Date'] =solarDf.apply(lambda x: changeFormate(x),axis=1)
solarDf=dataFilter(solarDf)

Combine these datasets into weather dataset

In [8]:
Temp1Df = maxTemDf.merge(minTemDf, on='Date')
Temp2Df = Temp1Df.merge(solarDf,on='Date')
Temp3Df = Temp2Df.merge(rainfallDf,on='Date')

In [9]:
toKeep=['Year','Month','Day','Date', 'Maximum temperature (Degree C)', 'Minimum temperature (Degree C)',
         'Daily global solar exposure (MJ/m*m)','Rainfall amount (millimetres)']

In [10]:
weatherDf = Temp3Df[toKeep].copy()

In [11]:
weatherDf.fillna({'Rainfall amount (millimetres)':0},inplace=True)

Calculate ETo of each day

In [12]:
weatherDf['EToi']=weatherDf.apply(lambda x:calculateEToi(x),axis=1)

In [13]:
#weatherDf.to_csv('outputData/weather.csv')

Get sum of ETo by month

In [14]:
grouping=['Year','Month']
group1=weatherDf.groupby(grouping).sum()

In [15]:
#group1.to_csv('outputData/ET.csv')

### irragetion water usage dataset

water usage.*

In [16]:
waterUsageMonthDf=pd.read_csv('dataset/waterUsagemonth.csv') 
ETDf=pd.read_csv('outputData/ET.csv') 
weatherDf=pd.read_csv('outputData/weather.csv')

In [17]:
ETDf['Month/Year'] =ETDf.apply(lambda x: changeFormate1(x),axis=1)
waterUsageMonthDf['Month/Year'] =waterUsageMonthDf.apply(lambda x: changeFormate1(x),axis=1)

In [18]:
toDrop=['Year','Month']
waterUsageMonthDf.drop(toDrop,axis=1,inplace=True)

In [19]:
Temp = waterUsageMonthDf.merge(ETDf, on='Month/Year')

In [20]:
#Temp.columns

In [21]:
toDrop=['CCD', 'DC800A', 'CODW', ' CODO', 'CCS','Maximum temperature (Degree C)', 'Minimum temperature (Degree C)',
       'Daily global solar exposure (MJ/m*m)', 'Rainfall amount (millimetres)','Unnamed: 17','Day','Month','Year']
Temp.drop(toDrop,axis=1,inplace=True)

In [22]:
weatherDf['Month/Year'] =weatherDf.apply(lambda x: changeFormate1(x),axis=1)

In [23]:
Temp1 = weatherDf.merge(Temp, on='Month/Year')
Temp1.head()

,Unnamed: 0,Year,Month,Day,Date,Maximum temperature (Degree C),Minimum temperature (Degree C),Daily global solar exposure (MJ/m*m),Rainfall amount (millimetres),EToi_x,...,rice_pro(%),rice_monthly_usage,maize_area(ha),maize_pro(%),maize_monthly_usage,wheat_area(ha),wheat_pro(%),wheat_monthly_usage,total_usage,EToi_y
0,181,2010,7,1,7/1/2010,13.2,3.5,9.6,0.0,0.733697,...,68.3,1692.474,4367,7.2,178.416,11334,5.1,126.378,2478,20.000339
1,182,2010,7,2,7/2/2010,8.7,4.8,5.0,0.2,0.227479,...,68.3,1692.474,4367,7.2,178.416,11334,5.1,126.378,2478,20.000339
2,183,2010,7,3,7/3/2010,12.1,1.8,9.8,6.4,0.730479,...,68.3,1692.474,4367,7.2,178.416,11334,5.1,126.378,2478,20.000339
3,184,2010,7,4,7/4/2010,9.5,1.0,9.8,0.0,0.618009,...,68.3,1692.474,4367,7.2,178.416,11334,5.1,126.378,2478,20.000339
4,185,2010,7,5,7/5/2010,12.3,1.6,6.2,0.4,0.471028,...,68.3,1692.474,4367,7.2,178.416,11334,5.1,126.378,2478,20.000339


In [24]:
#Temp1.columns

In [25]:
toDrop=['Unnamed: 0','total_usage']
Temp1.drop(toDrop,axis=1,inplace=True)

In [26]:
def riceDaily(df):
    df['riceDaily']=(df['EToi_x']/df['EToi_y'])*df['rice_monthly_usage']/df['rice_area(ha)']
    return df['riceDaily']

In [27]:
def maizeDaily(df):
    df['maizeDaily']=(df['EToi_x']/df['EToi_y'])*df['maize_monthly_usage']/df['maize_area(ha)']
    return df['maizeDaily']

In [28]:
def wheatDaily(df):
    df['wheatDaily']=(df['EToi_x']/df['EToi_y'])*df['wheat_monthly_usage']/df['wheat_area(ha)']
    return df['wheatDaily']

In [29]:
Temp1['riceDaily']=Temp1.apply(lambda x:riceDaily(x),axis=1)
Temp1['maizeDaily']=Temp1.apply(lambda x:maizeDaily(x),axis=1)
Temp1['wheatDaily']=Temp1.apply(lambda x:wheatDaily(x),axis=1)

In [30]:
#Temp1.columns

In [31]:
toDrop=[ 'EToi_x', 'Month/Year', 'rice_area(ha)', 'rice_pro(%)',
       'rice_monthly_usage', 'maize_area(ha)', 'maize_pro(%)',
       'maize_monthly_usage', 'wheat_area(ha)', 'wheat_pro(%)',
       'wheat_monthly_usage', 'EToi_y','Date']
Temp1.drop(toDrop,axis=1,inplace=True)

In [32]:
Temp1.head()

,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Daily global solar exposure (MJ/m*m),Rainfall amount (millimetres),riceDaily,maizeDaily,wheatDaily
0,2010,7,1,13.2,3.5,9.6,0.0,0.004278,0.001499,0.000409
1,2010,7,2,8.7,4.8,5.0,0.2,0.001326,0.000465,0.000127
2,2010,7,3,12.1,1.8,9.8,6.4,0.004260,0.001492,0.000407
3,2010,7,4,9.5,1.0,9.8,0.0,0.003604,0.001262,0.000345
4,2010,7,5,12.3,1.6,6.2,0.4,0.002747,0.000962,0.000263


### feature and lable 

In [33]:
featureDf=Temp1[['Month','Maximum temperature (Degree C)',
       'Minimum temperature (Degree C)',
       'Daily global solar exposure (MJ/m*m)', 'Rainfall amount (millimetres)']]

In [34]:
#featureDf.to_csv('outputData/dailyUsage/featureData.csv')

In [35]:
riceDaily=Temp1[['riceDaily']]
maizeDaily=Temp1[['maizeDaily']]
wheatDaily=Temp1[['wheatDaily']]

In [36]:
#wheatDaily.to_csv('outputData/dailyUsage/wheat_waterUsage.csv')
#riceDaily.to_csv('outputData/dailyUsage/riceDaily_waterUsage.csv')
#maizeDaily.to_csv('outputData/dailyUsage/maizeDaily_waterUsage.csv')